In [ ]:
# Import libraries
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

#### Data Collections

In [ ]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

print(df.head())


#### Data Review and Profiling

In [ ]:
missing_value = df.isnull().sum()
print("Missing Values:")
print(missing_value)

print("Unique Values:")
for col in df.columns:
    unique_values = df[col].nunique()
    print(f"{col}: {unique_values}")

print(f"Data Dimension (rows, columns): {df.shape})")

print("Data Information:")
df.info()

print("Data Description:")
print(df.describe())

#### Modeling

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # Input layer with appropriate input shape
    layers.Dense(32, activation='relu'), # Hidden layer
    layers.Dense(1, activation='sigmoid') # Output layer for binary classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', # Choose appropriate loss function for your task
    metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy Keras: {accuracy}")

# Make Predictions on the test data
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype("int32") # Convert probabilities to class labels

# Calculate the accuracy of the model
accuracy_sklearn = accuracy_score(y_test, y_pred_class)
print(f"Accuracy Sklearn: {accuracy_sklearn}")

# Generate a classifications report
report = classification_report(y_test, y_pred_class, zero_division=1)
print(f"Classification Report:")
print(report)

#### Model Visualization

In [ ]:
cm = confusion_matrix(y_test, y_pred_class)

plt.figure(figsize=(8, 6))
plt.tight_layout()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


#### Model Prediction

In [ ]:
# Make predictions on new data
X_new = pd.DataFrame(
    [[5.1, 3.5, 1.4, 0.2],
     [6.3, 3.3, 6.0, 2.5],
     [5.9, 3.0, 4.2, 1.5]],
    columns=iris.feature_names
)
print(X_new.head())

# Make predictions using the trained model
predictions = model.predict(X_new)
print(predictions)